In [ ]:
from gensim.models.keyedvectors import FastTextKeyedVectors
fasttext = FastTextKeyedVectors.load("D:/fasttext_word2vec/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model")


In [7]:
from text_processing import get_text_map
from os import listdir
from os.path import join
import random
import json
from collections import OrderedDict
import numpy as np
import pandas as pd

In [8]:
texts_lenta = pd.read_csv("./articles/music_lenta.csv")

In [11]:
texts_index = [0,
17,
26,
32,
98,
121,
130,133,
200,
231,240,316,331,334,336,366,371]

In [10]:
texts_index = [0,
17,
26,
32]

In [ ]:
text_maps_json_dict[0]

In [12]:
text_maps_json_dict = {}
for tex_ind in texts_index:
    text_map = get_text_map(texts_lenta.iloc[tex_ind]['text'], raw_text_input=True)
    text_maps_json_dict[tex_ind] = text_map

In [ ]:
text_maps_json_dict[371]['overall_colloc_text']

In [13]:
question_text_sent_collocind_dict = {"0":{'0':[9,11],'2':[26,35],'5':[13,27]},
                                     "17":{'0':[7,18],'2':[8,19],'4':[2,16],'8':[7,25],'10':[14,24],'12':[9,24]},
                                     "26":{'0':[6,19],'5':[5,11],'7':[0,6],'13':[13,27]},
                                     "32":{'3':[9,20],'5':[5,17],'14':[6,17],'18':[3,19]},
                                    "98":{'0':[11,16],'4':[3,6],'5':[1,12],'8':[7,12]},
                                    "121":{'0':[11,16],'1':[2,7],'3':[5,19],'4':[0,15],'4':[0,15],'6':[13,17],'11':[0,5]},
                                    "130":{'0':[17,28],'1':[0,7],'3':[2,9],'4':[1,8],'5':[6,10],'8':[15,19],'9':[12,28],'16':[0,12]},
                                    "133":{'2':[19,35],'6':[8,13],'8':[3,12],'9':[7,16],'18':[8,19],'20':[0,8],'20':[0,8],'40':[11,20],'49':[0,15]},
                                      "231":{'2':[0,7],'4':[0,19],'7':[0,12],'12':[13,30]},
                                    "240":{'1':[0,13],'11':[0,8],'15':[5,20]},
                                     "316":{'0':[12,20],'5':[8,13],'7':[9,20],'11':[20,27]},
                                     "331":{'2':[14,23],'4':[4,12],'10':[4,14]},"334":{'1':[4,10],'2':[0,3],'5':[4,12],'8':[1,9]},
                                     "336":{'1':[0,10],'7':[0,9],'10':[1,7],'13':[3,5]},"366":{'0':[0,4],'1':[0,12],'5':[15,24],'8':[8,20],'10':[0,15],'11':[0,18],'15':[2,11]},
                                     "371":{'0':[9,21],'2':[13,20],'3':[11,25],'6':[2,7],'12':[0,16],'21':[0,30]}}



In [ ]:
list(question_text_sent_collocind_dict.keys())

In [14]:
answers_dict = {}
for key in question_text_sent_collocind_dict.keys():
    answers_dict[key] = {}
    for setn_keys in question_text_sent_collocind_dict[key].keys():
        answers_dict[key][setn_keys] = True
answers_dict

{'0': {'0': True, '2': True, '5': True},
 '121': {'0': True, '1': True, '11': True, '3': True, '4': True, '6': True},
 '130': {'0': True,
  '1': True,
  '16': True,
  '3': True,
  '4': True,
  '5': True,
  '8': True,
  '9': True},
 '133': {'18': True,
  '2': True,
  '20': True,
  '40': True,
  '49': True,
  '6': True,
  '8': True,
  '9': True},
 '17': {'0': True, '10': True, '12': True, '2': True, '4': True, '8': True},
 '231': {'12': True, '2': True, '4': True, '7': True},
 '240': {'1': True, '11': True, '15': True},
 '26': {'0': True, '13': True, '5': True, '7': True},
 '316': {'0': True, '11': True, '5': True, '7': True},
 '32': {'14': True, '18': True, '3': True, '5': True},
 '331': {'10': True, '2': True, '4': True},
 '334': {'1': True, '2': True, '5': True, '8': True},
 '336': {'1': True, '10': True, '13': True, '7': True},
 '366': {'0': True,
  '1': True,
  '10': True,
  '11': True,
  '15': True,
  '5': True,
  '8': True},
 '371': {'0': True, '12': True, '2': True, '21': True, '

In [ ]:
effected_collocations_start_indexes_list = [i for i in range(effected_collocations_start_indexes_list[0],effected_collocations_start_indexes_list[1]+1)]

In [15]:
class user_vector:
    def __init__(self,debug = False):
        self.debug = debug
        self.vocab_features =[]
        self.sentence_features =[]
        #coreference_items, negation_items, sent_special_pos, dependencies_length, Y (answer)                   
        self.text_fearues = [] #OrderedDict([("lix",[]),("ttr",[])])
        self.answers_count = OrderedDict([("correct_answers",0),("incorrect_answers",0)])
        self.collocations_list = []
    def start_new_text(self):
        self.answers_count['correct_answers'] = 0
        self.answers_count['incorrect_answers'] = 0
        if self.debug:print("answers count has been reset", self.answers_count['correct_answers'], self.answers_count['incorrect_answers'])
    
    def end_text(self, text_map):
        if self.debug:
            print("\n========")
            print("SUM UP TEXT VALUES")
            print("========\n")
        correct_answers_rate = round(self.answers_count['correct_answers'] / (self.answers_count['correct_answers'] 
                                                                              + self.answers_count['incorrect_answers']),2)
        current_text_features = []
        if self.debug: 
            print("answers_count", self.answers_count)
        current_text_features.append(text_map['lix'])
        current_text_features.append(text_map['ttr'])
        current_text_features.append(text_map['sentences_count'])
        current_text_features.append(text_map['average_sentence_length'])
        #current_text_features.extend(text_map['vocab_properties'])
        current_text_features.extend(text_map['sent_properties'])
        current_text_features.append(correct_answers_rate)
        self.text_fearues.append(current_text_features)
        if self.debug: print("TEXT FEATURES",self.text_fearues)
    
    def update_vector_with_answer_sentence(self, sentence_map, effected_collocations_start_indexes_list, correct_answer):
        #update setnence and text features
        if self.debug:
            print("\n===NEW REPLY CALCULATION====")
            print("\n========")
            print("ADDING SENTENCE RESULTS")
            print("========\n")
            
        #update setnence and text features
        if correct_answer == True:
            answer_value = 1
            self.answers_count['correct_answers'] += 1
            if self.debug: print("Answer for this question is correct")
        else:
            answer_value = 0
            self.answers_count['incorrect_answers'] += 1
            if self.debug: print("Answer for this question is incorrect")
        if self.debug:print("check answers count", self.answers_count['correct_answers'], self.answers_count['incorrect_answers'])
        current_sentence_features = []
        current_sentence_features.append(sentence_map['spec_sentence_features']['negation'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['coreference'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['vozvr_verb'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['prich'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['deepr'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['case_complexity'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['mean_depend_length'])
        
        #current_sentence_features.extend(sentence_map['average_vocabulary'])
        current_sentence_features.append(answer_value)#target variable
        self.sentence_features.append(current_sentence_features)
        
        if self.debug: print("SENTENCE FEATURES", current_sentence_features)
        
        if self.debug:
            print("\n========")
            print("ADDING VOCABULARY RESULTS")
            print("========\n") 
            
        for word_w in sentence_map['collocation_vectors_list']:
            if self.debug:print(word_w[1][0])
            effected_collocations_start_indexes_list_ranged = [i for i in range(effected_collocations_start_indexes_list[0],effected_collocations_start_indexes_list[1]+1)]
            if word_w[0] in effected_collocations_start_indexes_list_ranged:
                current_word_features = []
                #current_word_features.append(str("local_freq_") + str(word_w[1][1]))
                current_word_features.append(word_w[1][1])
                #current_word_features.append(str("global_freq_mpi_ln_") + glob_freq_log)
                current_word_features.append(word_w[1][2])
                current_word_features.extend(word_w[2][0])
                current_word_features.append(answer_value)
                self.collocations_list.append(word_w[1][0])
                #print(current_word_features)
                self.vocab_features.append(current_word_features)
    def export_user_db(self, learner_id):
                
        words_db = np.array([np.array(word) for word in self.vocab_features])
        word_db_path = learner_id + '_word_db.csv'
        np.savetxt(word_db_path, words_db, delimiter=',') 
        
        sentence_db = np.array([np.array(sent) for sent in self.sentence_features])
        sentence_db_path = learner_id + '_sentence_db.csv'
        np.savetxt(sentence_db_path, sentence_db, delimiter=',') 
        
        text_db = np.array([np.array(text) for text in self.text_fearues])
        text_db_path = learner_id + '_text_db.csv'
        np.savetxt(text_db_path, text_db, delimiter=',') 
        
        with open("colloc_lit.txt", "w", encoding = "utf-8") as f:
            for colloc in self.collocations_list:
                f.write(colloc + '\n')

In [ ]:
text_maps_json_dict.keys()

In [ ]:
question_text_sent_collocind_dict['0']['0']

In [ ]:
text_maps_json_dict.keys()

In [16]:
DEBUG = True
user = user_vector(debug = DEBUG)
for text in text_maps_json_dict.keys():
    print(text)
    text_map = text_maps_json_dict[text]
    user.start_new_text()
    if str(text) in question_text_sent_collocind_dict:
        for question_sentence_index in question_text_sent_collocind_dict[str(text)].keys():

            user.update_vector_with_answer_sentence(text_map['sentences_map'][int(question_sentence_index)], 
                                            effected_collocations_start_indexes_list = question_text_sent_collocind_dict[str(text)][str(question_sentence_index)],
                                            correct_answer = answers_dict[str(text)][str(question_sentence_index)])

        user.end_text(text_map)

0
answers count has been reset 0 0

===NEW REPLY CALCULATION====

ADDING SENTENCE RESULTS

Answer for this question is correct
check answers count 1 0
SENTENCE FEATURES [0, 0, 0, 0, 0, 0, 0.2090909090909091, 1]

ADDING VOCABULARY RESULTS

расход на
проведение
музыкальный
конкурс
евровидение
в
киев
оказываться
предельно
высокий

===NEW REPLY CALCULATION====

ADDING SENTENCE RESULTS

Answer for this question is correct
check answers count 2 0
SENTENCE FEATURES [0.018518518518518517, 0, 0.021353453732970453, 0, 0, 0.042706907465940906, 0.34716981132075475, 1]

ADDING VOCABULARY RESULTS

украинец
возмущать
запредельный
гонорар
артист
ведущий
шоу
то что
почти
весь
тендер
проводить в
сжимать
срок
по
сокращенный
процедура
а также
явно
лишний
неоправданный
трата
в страна
где
идти
война
речь идти о
вооружать
конфликт
в
донбасс
примечание лентыру
большой часть
население
не хватать
деньги
даже
на
оплата
коммуналки
говорить в
заметка

===NEW REPLY CALCULATION====

ADDING SENTENCE RESULTS

Answer f

похороны
исполнитель
проводить
серия
благотворительный
концерт
в он
честь

SUM UP TEXT VALUES

answers_count OrderedDict([('correct_answers', 3), ('incorrect_answers', 0)])
TEXT FEATURES [[0.6118, 0.75, 0.1, 0.166, 0.2, 0.2, 0.024266827607358696, 0.03000229547540447, 0.0, 0.07131167644959036, 0.191510992676087, 1.0], [0.8013, 0.63, 0.14, 0.18071428571428574, 0.0, 0.07142857142857142, 0.015800820036400317, 0.0328465229137713, 0.004244871454581902, 0.1455018545198959, 0.2126376629537841, 1.0], [0.5788000000000001, 0.7, 0.15, 0.15866666666666668, 0.3333333333333333, 0.26666666666666666, 0.0313841893273231, 0.032255172907681075, 0.0, 0.13056425505926705, 0.2094257679815719, 1.0], [0.4068, 0.55, 0.33, 0.14181818181818182, 0.24242424242424243, 0.30303030303030304, 0.021535155564896605, 0.010710915298319728, 0.0, 0.05420704597769228, 0.19590366066250947, 1.0], [0.6818000000000001, 0.68, 0.11, 0.15545454545454546, 0.0, 0.09090909090909091, 0.01330650055880611, 0.01947401807081597, 0.0, 0.06277

In [17]:
user.export_user_db("big_musician")  